In [1]:
import json
import pickle
import random

import torch
from torch import nn, optim
from torch import autograd
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import torch.nn.utils.rnn as rnn_utils

import nltk
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.bleu_score import sentence_bleu
import time

from Vocab import Vocab
from LanguageModel import LanguageModel

import torch
torch.cuda.set_device(0)

print('import over')

import over


In [2]:
print(torch.__version__)

0.3.1.post2


In [69]:
#convert index to one-hot
#support batch
batch_size=2
vocab_size = 4
sents_len=6

a=torch.LongTensor(batch_size, sents_len)%(vocab_size-1)
b=torch.unsqueeze(a, dim=2)

one_hot = torch.FloatTensor(batch_size, sents_len, vocab_size).zero_()
one_hot.scatter_(2, b, 1)
one_hot = one_hot.transpose(1,2)

print(a.size())
print(one_hot.size())

print(a)
print(one_hot)

torch.Size([2, 6])
torch.Size([2, 4, 6])

 0  0  0  1  1  0
 0  0  1  1  2  0
[torch.LongTensor of size 2x6]


(0 ,.,.) = 
  1  1  1  0  0  1
  0  0  0  1  1  0
  0  0  0  0  0  0
  0  0  0  0  0  0

(1 ,.,.) = 
  1  1  0  0  0  1
  0  0  1  1  0  0
  0  0  0  0  1  0
  0  0  0  0  0  0
[torch.FloatTensor of size 2x4x6]



In [ ]:
#
a=Variable(torch.rand(2, 5))
labels=torch.LongTensor([2,3])
nll_loss = nn.NLLLoss()
log_softmax = nn.LogSoftmax(dim=1)
softmax = nn.Softmax(dim=1)
cross_entropy = nn.CrossEntropyLoss()

loss = nll_loss(log_softmax(a), Variable(labels))
print(loss)

loss = nll_loss(torch.log(softmax(a)), Variable(labels))
print(loss)

loss = cross_entropy(a, Variable(labels))
print(loss)

In [ ]:
#large matrix in gpu
#43800*50, float tensor
vocab_size = 43800
sents_len = 50
batch_size=10


for epoch in range(200):
    s=0
    enc_inputs = torch.LongTensor(batch_size, sents_len)%(vocab_size-1)
    enc_inputs = enc_inputs.unsqueeze(dim=2)
    one_hot = torch.FloatTensor(batch_size, sents_len, vocab_size).zero_()
    one_hot.scatter_(2, enc_inputs, 1)
    one_hot = one_hot.transpose(1,2)
    a = Variable(one_hot, requires_grad = 0).cuda()

    for ii in range(1):
        w=Variable(torch.rand(batch_size, sents_len), requires_grad=1).cuda()
        w=softmax(w).unsqueeze(dim=2)
        prob_copy = torch.bmm(a,w)
        s+=prob_copy
    
# for ii in range(61):
#     prob_copy=Variable(torch.rand(batch_size, vocab_size), requires_grad=1).cuda()
#     s+=prob_copy

In [29]:
#some bleu calculating
predicts='matthew justice is an english film and television producer ( -- death <low_freq> death ) can <split> he can a lead of the \'\' \'s formal be subject can be a lead - in to the \'\' subject \'s real , formal , or extended name '' .'
labels='matthew justice is an english film and television producer ( born 1st november 1965 ) . <split> matthew is a governor of the <low_freq> , has produced he can be a lead - in to the \'\' subject \'s real , formal , or extended name '' .'
predicts='matthew joshua estrada ( born october 20 , 1988 ) is an american football safety who is currently a free of the toronto argonauts of <split> he the , , he , <low_freq> was a a toronto argonauts of the canadian football league .'
labels='matthew joshua estrada ( born october 20 , 1988 ) is an american football safety who is currently a member of the toronto argonauts . <split> on march 29 , 2012 matt estrada signed with the toronto argonauts of the canadian football league .'
predicts='matthew k. miller ( born 1949 in cincinnati , ohio ) is an american voice . and ) and kermit host . actor producer . <split> he is also known as kermit miller and kermit <low_freq> .'
labels='matthew k. miller ( born 1949 in cincinnati , ohio ) is an american stage , film , and television actor and voice artist . <split> he is also known as kermit miller and kermit <low_freq> .'
predicts='matthew <low_freq> or <low_freq> ( greek : <low_freq> <low_freq> <low_freq> , '' <low_freq> <low_freq> <low_freq> , , c. greek 1325 - <low_freq> ) <low_freq> ) was <split> c. was byzantine emperor from <low_freq> to <low_freq> .'
labels='matthew <low_freq> or <low_freq> ( greek : <low_freq> <low_freq> <low_freq> , '' <low_freq> <low_freq> <low_freq> '' ) ( c. 1325 -- <low_freq> or <low_freq> ) . <split> he was byzantine emperor from <low_freq> to <low_freq> .'
predicts='in fact , west virginia is the site of the worst coal mining disaster to date . <split> the the <low_freq> mining disaster of <low_freq> , west virginia 6 december 1907 .'
labels='in fact , west virginia is the site of the worst coal mining disaster to date . <split> with the <low_freq> mine disaster of <low_freq> , west virginia 6 december 1907 .'
predicts='in fact , a new term , <split> <low_freq> has has been coined to describe an unhealthy obsession with eating health food .'
labels='in fact , a new term . <split> <low_freq> , has been coined to describe an unhealthy obsession with eating health food .'
predicts='in fact , templeton rye brand whiskey is distilled and aged in <low_freq> indiana by <low_freq> of indiana , . <low_freq> . . an . flavoring <split> the is combined with an '' alcohol flavoring agent '' from clarendon flavor engineers , and finally bottled at an iowa facility .'
labels='in fact , templeton rye brand whiskey is distilled and aged in <low_freq> indiana by <low_freq> of indiana utilizing a recipe shared with other brands . <split> it is combined with an '' alcohol flavoring agent '' from clarendon flavor engineers , and finally bottled at an iowa facility .'

a=predicts.split(' ')
b=labels.split(' ')
sentence_bleu([b], a, smoothing_function = SmoothingFunction(epsilon=0.001).method1)

0.7710406610294067

In [41]:
#tensor as index
batch_size=3
sents_len=4
vocab_size=6
logits=torch.rand(3,4,6)
labels=torch.LongTensor([[0,2,3,1],[1,5,3,2],[3,4,2,1]])

print(logits)
print(labels)

# print(logits)
pos_bias=torch.LongTensor([i*vocab_size for i in range(sents_len)]).view(1,-1)
pos_bias = pos_bias.expand(batch_size, pos_bias.size(1))
print(pos_bias.size())

batch_bias = torch.LongTensor([i*vocab_size*sents_len for i in range(batch_size)]).view(-1,1)
batch_bias = batch_bias.expand(batch_bias.size(0), sents_len)
print(batch_bias.size())

indices=labels+pos_bias+batch_bias
indices = indices.view(-1)
logits=logits.view(-1)
results=logits[indices]
results=results.view(batch_size, sents_len)

print(pos_bias)
print(batch_bias)
print(results)


(0 ,.,.) = 
  0.7708  0.2877  0.6474  0.5719  0.6309  0.3723
  0.4128  0.1720  0.8442  0.1937  0.7954  0.4892
  0.8378  0.3698  0.2211  0.8136  0.9741  0.6237
  0.7293  0.5401  0.0281  0.4767  0.2357  0.8523

(1 ,.,.) = 
  0.2411  0.5577  0.5717  0.9420  0.7847  0.2945
  0.7278  0.0168  0.4236  0.9483  0.1195  0.4750
  0.2507  0.6052  0.5003  0.3944  0.1121  0.3206
  0.7008  0.3773  0.9835  0.2848  0.2439  0.0223

(2 ,.,.) = 
  0.5443  0.2949  0.2276  0.2788  0.8578  0.7674
  0.5496  0.7211  0.6500  0.1105  0.8284  0.6783
  0.3357  0.5301  0.0776  0.6669  0.9750  0.4637
  0.7342  0.3033  0.0040  0.0271  0.5337  0.5884
[torch.FloatTensor of size 3x4x6]


 0  2  3  1
 1  5  3  2
 3  4  2  1
[torch.LongTensor of size 3x4]

torch.Size([3, 4])
torch.Size([3, 4])

  0   6  12  18
  0   6  12  18
  0   6  12  18
[torch.LongTensor of size 3x4]


  0   0   0   0
 24  24  24  24
 48  48  48  48
[torch.LongTensor of size 3x4]


 0.7708  0.8442  0.8136  0.5401
 0.5577  0.4750  0.3944  0.9835
 0.2